In [1]:
packages <- c("genomation","GenomicFeatures","glue","dplyr","Hmisc","GenomicRanges")
packageswarning <- sapply(packages,function(x){suppressMessages(library(x,character.only=TRUE))})
################################
findoverlapsin2grs <- function(first.gr,second.gr){
	hits <- findOverlaps(first.gr, second.gr,ignore.strand=TRUE)
	result <- data.frame(first.gr[queryHits(hits)],second.gr[subjectHits(hits)])
  return(result)
}

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”


In [2]:
promoters <- readRDS("promoters.gr.rds")
abund.df  <- readRDS("abund.df.rds")
genes_id_name <- readRDS("genes_id_name.rds")

In [3]:
log2_1 <- function(x){log2(x+1)}
# cal mean
mean_rmna <- function(x){mean(x,na.rm=TRUE)}
# in samples, corss genes
cal_matrix_cor_samples <- function(col,abund.df,overlap.samples.genemean.df){
    matrix <- na.omit(as.matrix(cbind(abund.df[,"FPKM"] %>% as.numeric,
                       overlap.samples.genemean.df[,col]%>% as.numeric) 
                       %>% na_if(-Inf)))
    genessnum <- nrow(matrix)
    if(genessnum <= 4) return(NULL)
    pearson <- rcorr(matrix,type="pearson")
    spearman <- rcorr(matrix,type="spearman")
    result <- cbind(methods=colnames(overlap.samples.genemean.df)[col],
          pearson=pearson$r[-1,1],
          spearman=spearman$r[-1,1],
          pearson_p=pearson$P[-1,1],
          spearman_p=spearman$P[-1,1])
    return(result)
}

In [4]:
cal_cor <- function(abund.df,overlap.samples.genemean.df,gene,method){
    abund.index <- match(genes_id_name$ref_gene_id[match(overlap.samples.genemean.df %>% pull(GENEID),genes_id_name$GENEID)],
    abund.df %>% pull(GENEID)) 
    abund.df <- abund.df[abund.index,]
    results.list <- lapply(2:ncol(overlap.samples.genemean.df),cal_matrix_cor_samples,abund.df=abund.df,
        overlap.samples.genemean.df=overlap.samples.genemean.df)
     results.df <- do.call(rbind,results.list) %>% data.frame %>% 
    mutate_at(c("pearson","spearman","pearson_p","spearman_p"),as.numeric) %>% 
    dplyr::select(c("methods","pearson","spearman","pearson_p","spearman_p"))
}

run_cal_cor <- function(norm_method,overlap.samples.genemean.df){
    # abund normalization
    if(norm_method=="log2_1"){
        abund.df <- abund.df %>% mutate_if(is.numeric,log2_1)
    }else{
    	 # not normlize
    	 abund.df <- abund.df
    }
    # calculate
    results.df <- cal_cor(abund.df=abund.df,overlap.samples.genemean.df=overlap.samples.genemean.df) #%>% na_if(-Inf) %>% na_if(Inf)
    return(results.df)
}   

In [5]:
methods <- c("CAMDA","CHALM","Meanm","Wemics")
data.gr <- readRDS("example.gr.rds")
overlap.samples.df <- findoverlapsin2grs(promoters,data.gr)
overlap.samples.genemean.df <- 
    overlap.samples.df %>% 
    dplyr::select(GENEID,all_of(methods)) %>% 
    group_by(GENEID) %>% 
    summarise_all(mean_rmna) %>% ungroup %>% data.frame
################################
sample_results <- run_cal_cor(norm_method="log2_1",overlap.samples.genemean.df=overlap.samples.genemean.df)


In [6]:
sample_results

methods,pearson,spearman,pearson_p,spearman_p
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
CAMDA,-0.3375031,-0.3761615,0,0
CHALM,-0.5027113,-0.5623298,0,0
Meanm,-0.4676377,-0.5391129,0,0
Wemics,-0.5454676,-0.5752751,0,0
